In [1]:
# Ollama 기본 셋팅
from langchain_ollama import ChatOllama
from langchain_core.prompts import( SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder )
from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
model = "llama3.2:1b"

llm = ChatOllama(base_url=base_url, model=model)

llm

ChatOllama(model='llama3.2:1b', base_url='http://localhost:11434')

In [2]:
# pdf 읽기
# pip install pymupdf tiktoken
import os
from langchain_community.document_loaders import PyMuPDFLoader

pdfs = []
docs = []

# 해당 폴더에 있는 모든 pdf 파일을 읽는다
for root, dirs, files in os.walk("rag-dataset/gym supplements"):
    # print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

for root, dirs, files in os.walk("rag-dataset/health supplements"):
    # print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))            

# 경로에 있는 각 pdf 내용을 리스트에 추가한다
for pdf_path in pdfs:
    loader = PyMuPDFLoader(pdf_path)
    temp = loader.load()
    docs.extend(temp)

print('pdf 갯수 : ',len(pdfs))

pdf 갯수 :  5


In [3]:
# 리스트에 있는 내용들을 하나의 문자열로 모두 합치는 함수
def format_docs(docs):
    return "\n\n".join([x.page_content for x in docs])

In [4]:
context = format_docs(docs)

In [5]:
# 프롬프트를 작성
system_prompt = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who works as document summarizer.
                                                          You must not hallucinate or generate false information.""")

prompt = """Summarize the given context in {words}.
### Context:
{context}

### Summary:"""

human_prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system_prompt, human_prompt]
template = ChatPromptTemplate(messages)

# 체인
summary_chain = template | llm | StrOutputParser()

In [6]:
response = summary_chain.invoke({'context':context, 'words': 500})
print(response)

The article discusses the potential health risks associated with using certain herbal supplements, particularly those that are commonly consumed as dietary addictions or over-the-counter products. These include:

1. **Vitamin and mineral deficiencies:** Black cohosh (black drink root), kava kava, saw palmetto, milk thistle, ginseng, garlic, and ginger have been associated with liver toxicity and other health issues.
2. **Liver damage:** Certain botanicals like saw palmetto, milk thistle, and ginkgo biloba may induce liver dysfunction or exacerbate pre-existing conditions such as hemochromatosis (iron overload).
3. **Cardiovascular risks:** Some herbal supplements like black cohosh, garlic, and ginkgo biloba have been linked to increased bleeding risk.
4. **Immune system suppression:** Certain herbs like saw palmetto, milk thistle, and ginseng may suppress the immune system or interact with immunosuppressive medications.

The article highlights the importance of being aware of potential

In [7]:
### pdf문서에서 보고서로 생성
# 프롬프트를 작성한다
system_prompt = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who answer user question based on the provided context.
                                                          Do not answer in more than {words} words""")

prompt = """Answer user question based on the provided context ONLY! If you do not know the answer, just say "I dont know".
### Context:
{context}

### Question:
{question}

### Answer:
"""

human_prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system_prompt, human_prompt]
template = ChatPromptTemplate(messages)

# 체인
qna_chain = template | llm | StrOutputParser()

In [8]:
question = "Provide a detailed report from the provided context. Write answer in Markdown."

response = qna_chain.invoke({'context':context, 'question':question, 'words':2000})

print(response)

**Detailed Report on Herbal Supplements and Their Potential Risks**

The report highlights various herbal supplements that have been associated with potential health risks, including liver toxicity, cardiovascular issues, and bleeding disorders. Here's a detailed breakdown of the information:

#### Black Cohosh

* **Potential risk:** Jaundice and liver failure in menopausal women
* **Mechanism:** Oxidative stress and decreased catalase activity
* **Case reports:** Several cases have been reported, including one case where a 71-year-old man experienced persistent surgical bleeding attributed to garlic ingestion.

#### Kava Kava

* **Potential risk:** Liver toxicity, which can require transplantation
* **Mechanism:** Depletion of glutathione and inhibition of cyclooxygenases
* **Case reports:** Several cases have been reported, including one case where a 67-year-old man experienced cholestatic hepatitis.

#### Saw Palmetto

* **Potential risk:** Cholestatic hepatitis and pancreatitis
* *